## Library Imports

In [ ]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix


# Suppress all pandas warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
df_fights = pd.read_csv('../datasets/raw/ufc-master.csv')
df_rankings = pd.read_csv('../datasets/raw/rankings_history.csv')
df_fights.head()
df_rankings.head()

In [ ]:
# Handle missing values in numeric columns
numeric_cols = df_fights.select_dtypes(include=[np.number]).columns
df_fights[numeric_cols] = df_fights[numeric_cols].fillna(0)

Machine learning models cannot process missing (NaN) values
For fight statistics, a missing value often indicates that the stat didn't occur, making 0 a logical replacement

In [ ]:
# Encode Winner column
df_fights['Winner'] = df_fights['Winner'].map({'Red': 1, 'Blue': 0})

## Feature Creation

In [ ]:
# Select relevant features
features = [
    # Fighter Records
    'RedWins', 'RedLosses', 'RedDraws',
    'BlueWins', 'BlueLosses', 'BlueDraws',
    
    # Win Streaks
    'RedCurrentWinStreak', 'BlueCurrentWinStreak',
    'RedLongestWinStreak', 'BlueLongestWinStreak',
    
    # Strike Statistics
    'RedAvgSigStrLanded', 'BlueAvgSigStrLanded',
    'RedAvgSigStrPct', 'BlueAvgSigStrPct',

    # Grappling Statistics
    'RedAvgTDLanded', 'BlueAvgTDLanded',
    'RedAvgTDPct', 'BlueAvgTDPct',
    'RedAvgSubAtt', 'BlueAvgSubAtt'
]

# Create feature matrix X
X = df_fights[features]

# Create target vector y
y = df_fights['Winner']

We're using a curated set of features that are most relevant for fight prediction and defining our feature matrix X and target vector y

## Model Parameters

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Model Creation and Training

In [ ]:
# Initialize RandomForest model with 100 trees for balance between complexity and performance
rf_model = RandomForestClassifier(
    n_estimators=100,  # Number of trees
    random_state=42    # Seed for reproducibility
)

# Train the model on scaled features
rf_model.fit(X_train_scaled, y_train)

## Model Evaluation

In [ ]:
# Make predictions
y_pred = rf_model.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.3f}")

# Calcuate Performanance
print("\nClassification Report:\n",classification_report(y_test, y_pred))

Initial Results Analysis:
The model achieves an accuracy of approximately 60% on the test set which is slightly better than random chance (50%).

## Model Improvement

### Feature Engineering

In [ ]:
# Calculate win percentages
df_fights['RedWinPercentage'] = df_fights['RedWins'] / (df_fights['RedWins'] + df_fights['RedLosses'] + df_fights['RedDraws'])
df_fights['BlueWinPercentage'] = df_fights['BlueWins'] / (df_fights['BlueWins'] + df_fights['BlueLosses'] + df_fights['BlueDraws'])

This feature provides:\
Overall career success rate\
More normalized view of fighter performance\
Accounts for different career lengths

In [ ]:
# Calculate striking differentials
df_fights['StrikingEfficiencyDiff'] = df_fights['RedAvgSigStrPct'] - df_fights['BlueAvgSigStrPct']
df_fights['StrikesLandedDiff'] = df_fights['RedAvgSigStrLanded'] - df_fights['BlueAvgSigStrLanded']

This captures:\
Technical striking advantage\
Volume striking advantage\
Overall striking dominance

In [ ]:
# Calculate grappling differentials
df_fights['TakedownEfficiencyDiff'] = df_fights['RedAvgTDPct'] - df_fights['BlueAvgTDPct']
df_fights['TakedownsLandedDiff'] = df_fights['RedAvgTDLanded'] - df_fights['BlueAvgTDLanded']

This captures:\
Wrestling effectiveness\
Control potential\
Grappling dominance

In [ ]:
# Calculate experience metrics
df_fights['ExperienceDiff'] = (df_fights['RedTotalRoundsFought'] - df_fights['BlueTotalRoundsFought'])
df_fights['TitleBoutExperienceDiff'] = (df_fights['RedTotalTitleBouts'] - df_fights['BlueTotalTitleBouts'])

This captures:\
Fight experience gap\
High-level competition experience\
Career longevity difference

In [ ]:
# Calculate finish rates
df_fights['RedFinishRate'] = (df_fights['RedWinsByKO'] + df_fights['RedWinsBySubmission']) / df_fights['RedWins']
df_fights['BlueFinishRate'] = (df_fights['BlueWinsByKO'] + df_fights['BlueWinsBySubmission']) / df_fights['BlueWins']

This shows:\
Finishing ability\
Fight-ending power\
Submission prowess

In [ ]:
# Clean up any infinity or NaN values created during calculations
df_fights = df_fights.replace([np.inf, -np.inf], 0)
df_fights = df_fights.fillna(0)

Clean up calculated feature columns by checking for problematic values like infinity or extremely large values and replacing inf and -inf with 0

## Improved Model Assessment

In [ ]:
enhanced_features = features + ['RedWinPercentage', 'BlueWinPercentage',
    'StrikingEfficiencyDiff', 'StrikesLandedDiff',
    'TakedownEfficiencyDiff', 'TakedownsLandedDiff',
    'ExperienceDiff', 'TitleBoutExperienceDiff',
    'RedFinishRate', 'BlueFinishRate'
]

# Create feature matrix X
X = df_fights[enhanced_features]

# Create target vector y
y = df_fights['Winner']

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Model Training and Fitting

In [ ]:
# Train Random Forest model with same paramaters
rf_model = RandomForestClassifier(
    n_estimators=100,  
    random_state=42
)
rf_model.fit(X_train_scaled, y_train)

### Model Prediction and Evaluation

In [ ]:
# Make predictions
y_pred = rf_model.predict(X_test_scaled)

# Print model performance
print(f"Model Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance analysis
feature_importance = pd.DataFrame({
    'feature': enhanced_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

In [ ]:
#Create Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Calculate metrics
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)  # True Positive Rate (model's ability to detect Red corner wins)
specificity = tn / (tn + fp)  # True Negative Rate (model's ability to detect Blue corner wins)
precision = tp / (tp + fp)  #Accuracy of predicted Red corner wins
f1 = 2 * (precision * sensitivity) / (precision + sensitivity) # Harmonic mean of precision and recall

# Display metrics
print("\nDetailed Metrics:")
print(f"True Negatives (Blue wins correctly predicted): {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives (Red wins correctly predicted): {tp}")
print(f"Sensitivity (Proportion of actual Red wins correctly identified): {sensitivity:.3f}")
print(f"Specificity (Proportion of actual Blue wins correctly identified): {specificity:.3f}")
print(f"Precision (Proportion of predicted Red wins that were correct): {precision:.3f}")
print(f"F1 Score (Overall model performance): {f1:.3f}")

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=["Blue (0)", "Red (1)"], yticklabels=["Blue (0)", "Red (1)"])
plt.title("Confusion Matrix for UFC Winner Prediction Model")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

Insights from Confusion Matrix:\
The F1 score of 0.700 indicates imbalanced performance\
High sensitivity (0.796) shows strong Red corner predictions\
Low specificity (0.333) shows a significant weakness in Blue corner predictions